**Date Created: 10/19/2025**

In [ ]:
import os
import json
import requests
import pandas as pd
from datetime import datetime
from mysql import connector
from dotenv import load_dotenv

In [113]:
load_dotenv() # this must be run in order to grant getenv function permission to access objects in .env file. 

True

In [ ]:
API_KEY = os.getenv("API_KEY")
API_HOST = os.getenv("API_HOST")

## GET Request (Extract)

In [ ]:
# API Request
url = f"https://api.eia.gov/v2/electricity/retail-sales/data/?api_key={API_KEY}&frequency=monthly&data[0]=customers&data[1]=price&data[2]=revenue&data[3]=sales&facets[stateid][]=CA&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000"
# Don't forget to add "api_key =" and "&" after you insert API KEY variable
# Full documentation here: https://www.eia.gov/opendata/documentation/APIv2.1.0.pdf

In [ ]:
# Storing the API response into a variable
response = requests.get(url)
# Printing response would return the 200 code
json_data = response.json() # accessing the json formatted data
json_data # examine what the data hierarchy looks like within the "response" key, where the data we want is imbedded

{'response': {'total': '1770',
  'dateFormat': 'YYYY-MM',
  'frequency': 'monthly',
  'data': [{'period': '2025-07',
    'stateid': 'CA',
    'stateDescription': 'California',
    'sectorid': 'ALL',
    'sectorName': 'all sectors',
    'customers': '16397392',
    'price': '30.04',
    'revenue': '6795.9437',
    'sales': '22624.37124',
    'customers-units': 'number of customers',
    'price-units': 'cents per kilowatt-hour',
    'revenue-units': 'million dollars',
    'sales-units': 'million kilowatt hours'},
   {'period': '2025-07',
    'stateid': 'CA',
    'stateDescription': 'California',
    'sectorid': 'TRA',
    'sectorName': 'transportation',
    'customers': '13',
    'price': '17.38',
    'revenue': '10.4139',
    'sales': '59.92411',
    'customers-units': 'number of customers',
    'price-units': 'cents per kilowatt-hour',
    'revenue-units': 'million dollars',
    'sales-units': 'million kilowatt hours'},
   {'period': '2025-07',
    'stateid': 'CA',
    'stateDescriptio

In [ ]:
json_data # this contains a lot of nested data, we need to specify what we want to access
json_data["response"]["data"] 
# each dictionary is one row of data

[{'period': '2025-07',
  'stateid': 'CA',
  'stateDescription': 'California',
  'sectorid': 'ALL',
  'sectorName': 'all sectors',
  'customers': '16397392',
  'price': '30.04',
  'revenue': '6795.9437',
  'sales': '22624.37124',
  'customers-units': 'number of customers',
  'price-units': 'cents per kilowatt-hour',
  'revenue-units': 'million dollars',
  'sales-units': 'million kilowatt hours'},
 {'period': '2025-07',
  'stateid': 'CA',
  'stateDescription': 'California',
  'sectorid': 'TRA',
  'sectorName': 'transportation',
  'customers': '13',
  'price': '17.38',
  'revenue': '10.4139',
  'sales': '59.92411',
  'customers-units': 'number of customers',
  'price-units': 'cents per kilowatt-hour',
  'revenue-units': 'million dollars',
  'sales-units': 'million kilowatt hours'},
 {'period': '2025-07',
  'stateid': 'CA',
  'stateDescription': 'California',
  'sectorid': 'RES',
  'sectorName': 'residential',
  'customers': '14450126',
  'price': '32.58',
  'revenue': '2693.30255',
  'sal

## Transform

In [118]:
rows = [] # creating an empty list that will hold a row for each item
column_names = ['period', 'stateid', 'stateDescription', 'sectorid', 'sectorName', 'customers', 'price', 'revenue', 'sales', 'customers-units', 'price-units', 'revenue-units']